<a href="https://colab.research.google.com/github/TOKTAK007/Image-Segment/blob/main/Lotteryrev10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import numpy as np
import torch

data = np.array([
    812519, 157196, 845093, 375805, 121789, 913106,
    613106, 484669, 943703, 929332, 331583, 436594,
    620405, 981417, 361807, 319196, 155012, 658642,
    395919, 970618, 737867, 61905, 98597, 944308,
    880159, 819068, 639235, 77258, 32761, 45037,
    386372, 578171, 70935, 114475, 46750, 910261,
    556725, 713517, 691861, 292972, 684579, 501272,
    100787, 472270, 890422, 835538, 424603, 912307,
    38495, 803628, 201303, 100994, 972661, 506404,
    286051, 837893, 244083, 999997, 945811, 569391,
    873286, 347258, 516967, 831567, 51095, 503446,
    875938, 781403, 589227, 491774, 510541, 529924,
    453522, 17223, 967375, 812564, 691197, 340388,
    798787, 775476, 387006, 369765, 943647, 174055,
    516461, 962526, 61324, 570331, 109767, 724628,
    345650, 74824, 967134, 197079, 735867, 356564
])

# set input and output sequence length
seq_len = 5

# create input sequences
input_seqs = []
for i in range(len(data)-seq_len):
    input_seq = data[i:i+seq_len]
    input_seqs.append(input_seq)

for seq in input_seqs:
    print(seq)


[812519 157196 845093 375805 121789]
[157196 845093 375805 121789 913106]
[845093 375805 121789 913106 613106]
[375805 121789 913106 613106 484669]
[121789 913106 613106 484669 943703]
[913106 613106 484669 943703 929332]
[613106 484669 943703 929332 331583]
[484669 943703 929332 331583 436594]
[943703 929332 331583 436594 620405]
[929332 331583 436594 620405 981417]
[331583 436594 620405 981417 361807]
[436594 620405 981417 361807 319196]
[620405 981417 361807 319196 155012]
[981417 361807 319196 155012 658642]
[361807 319196 155012 658642 395919]
[319196 155012 658642 395919 970618]
[155012 658642 395919 970618 737867]
[658642 395919 970618 737867  61905]
[395919 970618 737867  61905  98597]
[970618 737867  61905  98597 944308]
[737867  61905  98597 944308 880159]
[ 61905  98597 944308 880159 819068]
[ 98597 944308 880159 819068 639235]
[944308 880159 819068 639235  77258]
[880159 819068 639235  77258  32761]
[819068 639235  77258  32761  45037]
[639235  77258  32761  45037 386372]
[



---

input_seq = data[i:i+seq_len] หมายถึงการสร้างชุดข้อมูล input โดยเริ่มจากข้อมูลที่ตำแหน่ง i ไปจนถึงตำแหน่ง i+seq_len-1 ซึ่ง seq_len คือความยาวของชุดข้อมูล input ที่ต้องการสร้าง เพื่อใช้ในการฝึกโมเดล LSTM โดยที่จะใช้ชุดข้อมูลนี้เป็นข้อมูล input สำหรับโมเดล LSTM แต่ละตัว ตัวอย่างเช่น ถ้า seq_len=5 และข้อมูลตัวอย่างคือ [1,2,3,4,5,6,7,8,9] จะได้ชุดข้อมูล input ดังนี้


---
[1,2,3,4,5]

[2,3,4,5,6]

[3,4,5,6,7]

[4,5,6,7,8]

[5,6,7,8,9]


---




In [130]:
# create output sequences
output_seqs = data[seq_len:]
print(output_seqs[:10])

[913106 613106 484669 943703 929332 331583 436594 620405 981417 361807]


In [131]:

# convert input_seqs and output_seqs to numpy arrays
input_seqs = np.array(input_seqs)
output_seqs = np.array(output_seqs)

# convert numpy arrays to PyTorch tensors
input_seqs = torch.from_numpy(input_seqs).float()
output_seqs = torch.from_numpy(output_seqs).float()

# reshape input_seqs to match expected input shape of LSTM (batch_size, seq_len, input_size)
input_seqs = input_seqs.reshape(-1, seq_len, 1)

# print shapes of input_seqs and output_seqs
print("input_seqs shape:", input_seqs.shape)
print("output_seqs shape:", output_seqs.shape)


input_seqs shape: torch.Size([91, 5, 1])
output_seqs shape: torch.Size([91])


In [132]:
print(input_seqs[:5])

tensor([[[812519.],
         [157196.],
         [845093.],
         [375805.],
         [121789.]],

        [[157196.],
         [845093.],
         [375805.],
         [121789.],
         [913106.]],

        [[845093.],
         [375805.],
         [121789.],
         [913106.],
         [613106.]],

        [[375805.],
         [121789.],
         [913106.],
         [613106.],
         [484669.]],

        [[121789.],
         [913106.],
         [613106.],
         [484669.],
         [943703.]]])


In [133]:
print(output_seqs[:5])

tensor([913106., 613106., 484669., 943703., 929332.])




---
.reshape() เป็น method ของ numpy array ที่ใช้สำหรับเปลี่ยนรูปแบบของ array โดยจะระบุขนาดของ array ใหม่ด้วย โดยการใช้ -1 แทนขนาดที่ไม่ต้องการระบุ ในที่นี้ input_seqs มีขนาด (num_seqs, seq_len) แต่ LSTM ของ PyTorch ต้องการข้อมูลในรูปแบบ (num_seqs, seq_len, input_dim) 


---


โดย input_dim คือจำนวน feature ของแต่ละ time step ใน input sequence ในที่นี้เป็น 1 เพราะ input sequence มีแค่ค่าเดียวในแต่ละ time step จึงจำเป็นต้องใช้ .reshape() เพื่อเปลี่ยนรูปแบบของ input_seqs ให้เป็น (num_seqs, seq_len, 1) โดยใช้ -1 แทน num_seqs และ 1 ในการระบุขนาดใหม่ของ input_seqs


---




In [134]:
import numpy as np
import torch
import torch.nn as nn

#สร้างโมเดล LSTM
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):     #LSTM ของ PyTorch ต้องการข้อมูลในรูปแบบ (num_seqs, seq_len, input_dim)
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, 1)       #โดยมีจำนวน input features เป็น hidden_size และจำนวน output features เป็น 1 
                                              #โดยค่่า output ที่ออกมาจะมีค่าเป็น (batch_size, 1)
  def forward(self, x):
      h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      out, _= self.lstm(x, (h0, c0))          # __ แทน h0 และ c0 ที่เก็บไว้ใช้ในการคำนวณต่อไป  #output tensor ขนาด (sequence_length, batch_size, hidden_size)
      out = self.fc(out[:, -1, :])
      return out

In [135]:
#กำหนด parameter
input_size = 1
hidden_size = 64
num_layers = 2
learning_rate = 0.001
num_epochs = 1000

#สร้างโมเดล LSTM
model = LSTM(input_size, hidden_size, num_layers)



---
h0 ได้ถูกต้องต้องระบุขนาด dimension แรกของ Tensor h0 ให้เท่ากับ num_layers และขนาด dimension ที่สองของ Tensor h0 ให้เท่ากับ x.size(0) ซึ่งเป็นจำนวนตัวอย่างในชุดข้อมูลนั้น


---






---
out = self.fc(out[:, -1, :])
จากนั้น out จะถูกส่งต่อไปยังชั้น Linear ด้วยเมท็อด fc ซึ่งเป็นชั้นเสริมเพื่อปรับขนาดของ tensor จาก (batch_size, hidden_size) เป็น (batch_size, 1) โดยการคูณกับเมทริกซ์เส้นเดียว (vector) ขนาด hidden_size ด้วยหลักการของหน่วยประมวลผลเชิงเส้น (linear transformation) ซึ่งในที่นี้คือการคูณกับเมทริกซ์ W ของชั้น Linear แล้วเทียบผลรวมกับเวกเตอร์ b ของชั้น Linear ด้วยการบวกเพื่อสร้าง output tensor ขนาด (batch_size, 1)


---
ดังนั้น เมื่อเราเลือกเอา output tensor จากโมเดล LSTM ใน time step สุดท้ายของแต่ละตัวอย่าง out[:, -1, :] จะเป็นการเลือกเฉพาะ dimension ที่สองของ tensor (dimension ที่เป็น batch_size) และ dimension ที่สามของ tensor (dimension ที่เป็น hidden_size) เพื่อให้ได้ tensor ขนาด (batch_size, hidden_size) ที่เก็บข้อมูล output ของโมเดล LSTM ใน time step สุดท้ายของแต่ละตัวอย่างในชุดข้อมูลเท่านั้น


---







---
การกำหนด batch_first=True หมายความว่าข้อมูลนำเข้าจะมีขนาดเป็น (batch_size, sequence_length, input_size) โดยที่ batch_size คือจำนวนตัวอย่างในแต่ละจุดเวลาและ 


---


sequence_length คือความยาวของชุดข้อมูลในแต่ละตัวอย่าง ซึ่งใช้สำหรับการแบ่งชุดข้อมูลเป็นชุดเล็กๆ (minibatch) เพื่อให้การคำนวณในการฝึกแบบจำลอง LSTM สามารถทำได้เร็วขึ้นโดยใช้ GPU และให้ผลลัพธ์เป็น (batch_size, sequence_length, hidden_size) ซึ่งเป็นขนาดที่สามารถนำไปใช้ต่อได้ในชุดข้อมูลถัดไปของโมเดล LSTM


---



In [136]:
import torch.optim as optim

#กำหนดค่า Loss และ Optimization
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [137]:
#Train Model
for epoch in range(num_epochs):
  X = input_seqs
  Y = output_seqs
  inputs = X
  targets = Y

  #เริ่มต้นการฝึก Model
  optimizer.zero_grad()
  Y_pred = model(inputs)

  #Cal Loss
  loss = criterion(Y_pred, Y)

  #คำนวณ gradient และอัปเดต weights
  loss.backward()
  optimizer.step()
  if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/1000], Train Loss: 377586155520.0000
Epoch [200/1000], Train Loss: 377578356736.0000
Epoch [300/1000], Train Loss: 377570951168.0000
Epoch [400/1000], Train Loss: 377563742208.0000
Epoch [500/1000], Train Loss: 377556598784.0000
Epoch [600/1000], Train Loss: 377549553664.0000
Epoch [700/1000], Train Loss: 377542508544.0000
Epoch [800/1000], Train Loss: 377535496192.0000
Epoch [900/1000], Train Loss: 377528483840.0000
Epoch [1000/1000], Train Loss: 377521471488.0000
